In [1]:
import numpy as np
import pandas as pd
import re
import time
import matplotlib as plt
%matplotlib inline
import seaborn as sns

In [2]:
sample_submission = pd.read_csv('sample_submission.csv')
train_data = pd.read_csv('train_2016_v2.csv')
properties_data= pd.read_csv('properties_2016.csv')
zillow_dict = pd.read_excel('zillow_data_dictionary.xlsx', sheetname=None)

/Users/kaijiang/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
con_columns = []
cat_columns = []
text_columns = []
id=['parcelid']


for i in range(len(zillow_dict['Data Dictionary'].variable)):
    if zillow_dict['Data Dictionary'].variable[i]=='con':
        con_columns.append(zillow_dict['Data Dictionary'].Feature[i].replace("'",""))
    if zillow_dict['Data Dictionary'].variable[i]=='cat':
        cat_columns.append(zillow_dict['Data Dictionary'].Feature[i].replace("'",""))
    if zillow_dict['Data Dictionary'].variable[i]=='text':
        text_columns.append(zillow_dict['Data Dictionary'].Feature[i].replace("'",""))

In [4]:
# then read the rule and join to get full data, then use simple random forest to get predicted results, 
# results same for all time points
groundtruth = train_data.merge(properties_data,right_on='parcelid',left_on='parcelid', how='left')
full_data = train_data.merge(properties_data,right_on='parcelid',left_on='parcelid', how='right')



In [5]:
full_data = full_data.replace(np.nan, -1)
full_data['taxdelinquencyflag']=full_data['taxdelinquencyflag'].replace('Y', 1)

In [ ]:
from sklearn import preprocessing
for i in cat_columns:
    
    le = preprocessing.LabelEncoder()
    le.fit(full_data[i])
    labels = dict(zip(full_data[i].unique()
               , le.transform(full_data[i].unique())))
    print(i)
    print(labels)
    #print(le.classes_)
    full_data[i]=le.transform(full_data[i])

airconditioningtypeid
{1.0: 1, -1.0: 0, 5.0: 3, 13.0: 7, 11.0: 5, 9.0: 4, 3.0: 2, 12.0: 6}
architecturalstyletypeid
{-1.0: 0, 7.0: 4, 8.0: 5, 2.0: 1, 10.0: 6, 3.0: 2, 21.0: 7, 5.0: 3, 27.0: 8}
buildingclasstypeid
{-1.0: 0, 4.0: 4, 3.0: 3, 5.0: 5, 2.0: 2, 1.0: 1}
decktypeid
{-1.0: 0, 66.0: 1}
fireplaceflag
{-1: 0, True: 1}


In [ ]:
OHE = preprocessing.OneHotEncoder()
full_data_ohe=OHE.fit_transform(full_data[cat_columns])

In [ ]:
SSL = preprocessing.StandardScaler()
for num_col in con_columns:
    full_data[num_col] = SSL.fit_transform(full_data[num_col])

In [ ]:
from scipy.stats import skew, boxcox
for i in con_columns:
    
    full_data[i], _ = boxcox(full_data[i]+20)

In [ ]:
from sklearn import grid_search, metrics

#scorer = metrics.make_scorer('ndcg_scorer', needs_proba=True) 
# 如果你要customize scorer

def search_model(train_x, train_y, est, param_grid, n_jobs, cv, refit=False):
##Grid Search for the best model
    model = grid_search.GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring = 'neg_mean_absolute_error',
                                     verbose  = 1,
                                     n_jobs  = n_jobs,
                                     iid    = True,
                                     refit    = refit,
                                     cv      = cv)
    # Fit Grid Search Model
    model.fit(train_x, train_y)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:", model.best_params_)
    print("Scores:", model.grid_scores_)
    return model

In [ ]:
import xgboost as xgb
param_grid = {
              'learning_rate':[0.02],
              'n_estimators':[500],
              'max_depth': [9],
              'min_child_weight':[50],
              'subsample': [0.78],
              'colsample_bytree':[0.67],
              'gamma':[0.9],
              'nthread': [-1],
              'seed' : [1234]}

model = search_model(X_train,
                     y_train,
                     xgb.XGBRegressor(),
                     param_grid,
                     n_jobs = 1,
                     cv = 4,
                     refit = True)

In [ ]:
y_predict = linear_reg.predict(X_test)
metrics.mean_absolute_error(y_test, y_predict)

In [ ]:
# in this way we get a baseline xgboost model score